# 1. gradio로 애플, 마이크로소프트, 테슬라, 엔비디아 주식 - yahoo finance에서 2014-전날까지 불러오기
# 2. 데이터 표시
# 3. 종가를 타겟 변수로 지정
# 4. AutoGluon을 이용해 시계열 분석
# 5. 30일 예측 결과 출력, 시각화

In [32]:
# !pip install yfinance gradio autogluon

In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import yfinance as yf
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [64]:
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]

- 이거는 여러가지 데이터를 분석할 때 필요한 함수임

In [66]:
# # 데이터를 합치고 모든 컬럼을 포함
# def prepare_data(stock_data):
#     data_ticker = pd.DataFrame()
#     for ticker, data in stock_data.items():
#         data['Ticker'] = ticker  # 종목명을 추가
#         data_ticker = pd.concat([data_ticker, data])  # 모든 데이터를 결합
#     return data_ticker

In [3]:
interface.close()

Closing server running on port: 7860


In [5]:
interface.close()

Closing server running on port: 7860


In [6]:
import pandas as pd
import gradio as gr
import yfinance as yf
import matplotlib.pyplot as plt
from autogluon.tabular import TabularPredictor
from datetime import datetime, timedelta

# 종목 리스트
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]

# Yahoo Finance에서 주식 데이터 가져오기
def fetch_stock_data(tickers):
    stock_data = {}
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    
    for ticker in tickers:
        stock_data[ticker] = yf.download(ticker, start="2014-01-01", end=end_date)
    return stock_data

# 사용자가 선택한 종목의 데이터를 추출
def select_stock_data(stock_data, selected_ticker):
    if selected_ticker not in stock_data:
        raise KeyError(f"Selected ticker {selected_ticker} not found in stock data")
    return stock_data[selected_ticker]

# AutoML을 통해 시계열 예측 수행 (종가를 타겟 변수로 지정)
def automl_time_series(data, time_limit=300):
    target_column = "Close"
    
    # 시간 기반으로 데이터를 분할
    train_data = data.iloc[:-30].copy()  # 30일 전까지 훈련 데이터로 설정 (copy() 사용)
    test_data = data.iloc[-30:].copy()   # 마지막 30일을 테스트 데이터로 설정 (copy() 사용)
    
    # Date 컬럼을 추가할 때 loc[] 사용하여 할당
    train_data.loc[:, 'Date'] = train_data.index
    test_data.loc[:, 'Date'] = test_data.index

    # AutoGluon 모델 학습
    model = TabularPredictor(label=target_column, problem_type='regression').fit(train_data, time_limit=time_limit)
    
    # 30일 예측
    pred = model.predict(test_data)
    
    # 여러 모델 성능 비교
    leader_board = model.leaderboard(test_data, silent=True)
    
    # 중요 변수 출력
    feature_importance = model.feature_importance(test_data, silent=True)
    
    prediction_df = pd.DataFrame({"Date": test_data['Date'], "Actual": test_data[target_column], "Predicted": pred})
    
    return {
        "predictions": prediction_df,
        "leader_board": leader_board,
        "feature_importance": feature_importance
    }

# 시각화 함수: 예측 결과
def plot_predictions(predictions):
    plt.figure(figsize=(10, 6))
    plt.plot(predictions['Date'], predictions['Actual'], label='Actual', marker='o')
    plt.plot(predictions['Date'], predictions['Predicted'], label='Predicted', linestyle='--', marker='x')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Actual vs Predicted Close Prices')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    # 이미지로 저장
    plt.savefig('predictions_plot.png')
    plt.close()
    return 'predictions_plot.png'

# 시각화 함수: 변수 중요도
def plot_feature_importance(feature_importance):
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance.index, feature_importance['importance'], color='lightgreen')
    plt.xlabel('Importance')
    plt.title('Feature Importance')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('feature_importance_plot.png')
    plt.close()
    return 'feature_importance_plot.png'

# 시각화 함수: 모델 성능 비교
def plot_leaderboard(leader_board):
    plt.figure(figsize=(10, 6))
    plt.barh(leader_board['model'], leader_board['score_test'], color='lightblue')
    plt.xlabel('Test Score (RMSE)')
    plt.title('Model Performance Comparison')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('leaderboard_plot.png')
    plt.close()
    return 'leaderboard_plot.png'

# 시각화 함수: 4개 종목 종가 그래프
def plot_multiple_stocks(stock_data):
    plt.figure(figsize=(12, 6))
    
    for ticker in tickers:
        plt.plot(stock_data[ticker].index, stock_data[ticker]['Close'], label=ticker)
    
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Stock Close Prices (AAPL, MSFT, TSLA, NVDA)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('multiple_stocks_plot.png')
    plt.close()
    return 'multiple_stocks_plot.png'

# 매수/매도 결정 함수 (간단한 가격 비교로 결정)
def make_trade_decision(predictions):
    last_actual_price = predictions['Actual'].iloc[-1]  # 마지막 실제 종가
    last_predicted_price = predictions['Predicted'].iloc[-1]  # 마지막 예측된 종가

    if last_predicted_price > last_actual_price:
        return "매수 신호: 예측된 가격이 상승할 것으로 보입니다."
    elif last_predicted_price < last_actual_price:
        return "매도 신호: 예측된 가격이 하락할 것으로 보입니다."
    else:
        return "유지 신호: 예측된 가격에 큰 변화가 없습니다."

# 분석 함수
def analyze_stock(selected_ticker):
    stock_data = fetch_stock_data(tickers)
    selected_data = select_stock_data(stock_data, selected_ticker)
    
    # AutoML을 통해 종가 예측 및 추가 정보 추출
    results = automl_time_series(selected_data)

    # 예측, 리더보드, 변수 중요도 각각 시각화
    prediction_plot = plot_predictions(results['predictions'])
    leaderboard_plot = plot_leaderboard(results['leader_board'].head(5))
    feature_importance_plot = plot_feature_importance(results['feature_importance'].head(5))
    
    # 4개 종목 모두를 보여주는 그래프 추가
    multiple_stocks_plot = plot_multiple_stocks(stock_data)

    # 매수/매도 신호 계산
    trade_signal = make_trade_decision(results['predictions'])

    return prediction_plot, leaderboard_plot, feature_importance_plot, trade_signal, multiple_stocks_plot

# Gradio 인터페이스 구성
with gr.Blocks() as interface:
    with gr.Row():
        with gr.Column():
            # 종목 선택 드롭다운 (기본값 설정)
            stock_selector = gr.Dropdown(choices=tickers, label="주식 종목 선택", value="AAPL")  # 기본값 AAPL로 설정
            run_button = gr.Button("30일 예측 및 시각화")

    with gr.Row():
        # 예측 결과 시각화 이미지 출력
        with gr.Column():
            feature_importance_image_output = gr.Image(label="Feature Importance(변수 중요도)")
        with gr.Column():
            leaderboard_image_output = gr.Image(label="Model Performance(모델 성능)")
        with gr.Column():
            prediction_image_output = gr.Image(label="Actual vs Predicted (실제값 VS 예측값)")

    with gr.Row():
        # 매수/매도 신호 출력
        trade_signal_output = gr.Textbox(label="매수/매도 신호")

    with gr.Row():
        # 4개 종목 종가 그래프 출력
        multiple_stocks_output = gr.Image(label="4개 종목 종가 그래프")

    # 버튼 클릭 시 분석 실행
    def run_analysis(selected_ticker):
        try:
            prediction_plot, leaderboard_plot, feature_importance_plot, trade_signal, multiple_stocks_plot = analyze_stock(selected_ticker)
        except Exception as e:
            return gr.update(value=f"Error during analysis: {e}"), "", "", "", "", ""

        # 분석 결과 반환 (이미지 파일 경로 및 매수/매도 신호)
        return prediction_plot, leaderboard_plot, feature_importance_plot, trade_signal, multiple_stocks_plot

    # 버튼 클릭 시 실행 및 업데이트
    run_button.click(
        run_analysis,
        inputs=[stock_selector],
        outputs=[prediction_image_output, leaderboard_image_output, feature_importance_image_output, trade_signal_output, multiple_stocks_output]
    )

# Gradio 앱 실행
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
No path specified. Models will be saved in: "AutogluonModels/ag-20241017_070503"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
Memory Avail:       9.55 GB / 15.31 GB (62.4%)
Disk Space Avail:   36.69 GB / 237.85 GB (15.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize

[1000]	valid_set's rmse: 3.04117
[2000]	valid_set's rmse: 2.81028
[3000]	valid_set's rmse: 2.71851
[4000]	valid_set's rmse: 2.66339
[5000]	valid_set's rmse: 2.63655
[6000]	valid_set's rmse: 2.60874
[7000]	valid_set's rmse: 2.59619
[8000]	valid_set's rmse: 2.58846
[9000]	valid_set's rmse: 2.58245
[10000]	valid_set's rmse: 2.57679


	-2.5768	 = Validation score   (-root_mean_squared_error)
	6.62s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 292.46s of the 292.46s of remaining time.


[1000]	valid_set's rmse: 1.43081
[2000]	valid_set's rmse: 1.41178


	-1.4052	 = Validation score   (-root_mean_squared_error)
	1.61s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 290.7s of the 290.7s of remaining time.
	-0.585	 = Validation score   (-root_mean_squared_error)
	0.73s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 289.43s of the 289.42s of remaining time.
	-1.7729	 = Validation score   (-root_mean_squared_error)
	65.19s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 224.13s of the 224.12s of remaining time.
	-0.7159	 = Validation score   (-root_mean_squared_error)
	0.67s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 222.95s of the 222.94s of remaining time.
	-3.0141	 = Validation score   (-root_mean_squared_error)
	2.94s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost .

In [7]:
interface.close()

Closing server running on port: 7860
